In [11]:
# !pip install tqdm

import requests
import pandas as pd
import time
from tqdm import tqdm

BASE_URL = "https://www.imf.org/external/datamapper/api/v1"

# Define indicator codes and their readable names
INDICATOR_IDS = {
    "rev": "Government revenue (% of GDP)",
    "exp": "Government expenditure (% of GDP)",
    "prim_exp": "Primary expenditure (% of GDP)",
    "ie": "Interest on public debt (% of GDP)",
    "pb": "Primary balance (% of GDP)",
    "d": "Gross public debt (% of GDP)",
    "rgc": "Real GDP growth rate (%)",
    "rltir": "Real long-term bond yield (%)"
}

# Optional: restrict to years
YEAR_FILTER = []  # e.g., [2015, 2016, 2017]

def fetch_country_names():
    url = f"{BASE_URL}/countries"
    response = requests.get(url)
    response.raise_for_status()
    raw = response.json()
    return {
        k: v.get('label', k) for k, v in raw.items()
        if isinstance(v, dict) and 'label' in v
    }

def fetch_timeseries(indicator_code, retries=3, delay=5):
    url = f"{BASE_URL}/{indicator_code}"
    if YEAR_FILTER:
        year_param = ",".join(map(str, YEAR_FILTER))
        url += f"?periods={year_param}"

    for attempt in range(retries):
        try:
            response = requests.get(url)
            response.raise_for_status()
            values = response.json()["values"]

            # Handle two formats:
            # 1. {country_code: {year: value, ...}, ...}
            # 2. {indicator_code: {country_code: {year: value, ...}, ...}}
            if indicator_code in values:
                return values[indicator_code]  # Nested case
            return values  # Flat case

        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {indicator_code}: {e}")
            time.sleep(delay)
    raise Exception(f"Failed to fetch {indicator_code} after {retries} retries")


def fetch_all_data(indicators):
    rows = []
    for code, label in tqdm(indicators.items(), desc="Fetching indicators"):
        raw = fetch_timeseries(code)
        for country_code, yearly_data in raw.items():
            for year, value in yearly_data.items():
                if YEAR_FILTER and int(year) not in YEAR_FILTER:
                    continue
                rows.append({
                    "year": int(year),
                    "country_code": country_code,
                    "indicator": label,
                    "value": value
                })
    return pd.DataFrame(rows)

if __name__ == "__main__":
    print("Fetching country names...")
    country_names = fetch_country_names()

    print("Downloading data...")
    df_long = fetch_all_data(INDICATOR_IDS)

    print("Mapping country names...")
    df_long["country"] = df_long["country_code"].map(lambda x: country_names.get(x, x))
    df_long = df_long[["year", "country", "indicator", "value"]]
    df_long = df_long.sort_values(["country", "indicator", "year"])


    df_long.to_csv("imf_macro_indicators_long.csv", index=False)
    print("✅ Saved to imf_macro_indicators_long.csv")



Fetching country names...


Fetching indicators: 100%|██████████| 8/8 [00:26<00:00,  3.28s/it]

Mapping country names...
✅ Saved to imf_macro_indicators_long.csv


In [24]:
import requests
import pandas as pd

def fetch_country_dataframe():
    """
    Fetches country data from the IMF API and returns it as a Pandas DataFrame.
    The DataFrame will have 'country_code' and 'country_name' columns.
    """
    BASE_URL = "https://www.imf.org/external/datamapper/api/v1"
    url = f"{BASE_URL}/countries"

    try:
        # Make the GET request to the API
        response = requests.get(url)
        # Raise an HTTPError for bad responses (4xx or 5xx)
        response.raise_for_status()
        # Parse the JSON response
        raw_data = response.json()

        # Extract the 'countries' dictionary from the response
        countries_data = raw_data.get("countries", {})

        # Prepare a list to hold the data for the DataFrame
        country_list = []
        for code, details in countries_data.items():
            # Ensure 'details' is a dictionary and has a 'label' key
            if isinstance(details, dict) and "label" in details:
                country_list.append({
                    "country_code": code,
                    "country_name": details["label"]
                })
            else:
                # Handle cases where 'label' might be missing or details is not a dict
                print(f"Warning: Skipping country code '{code}' due to unexpected data format: {details}")

        # Create a Pandas DataFrame from the list of dictionaries
        df = pd.DataFrame(country_list)
        return df

    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from IMF API: {e}")
        return pd.DataFrame() # Return an empty DataFrame on error
    except ValueError as e:
        print(f"Error parsing JSON response: {e}")
        return pd.DataFrame() # Return an empty DataFrame on error

if __name__ == "__main__":
    print("Fetching country data and creating DataFrame...")
    country_df = fetch_country_dataframe()

    if not country_df.empty:
        print("\nDataFrame Head:")
        print(country_df.head())

        print(f"\nTotal countries fetched: {len(country_df)}")

        # Save the DataFrame to a CSV file
        output_filename = "imf_countries.csv"
        country_df.to_csv(output_filename, index=False)
        print(f"\nDataFrame saved to {output_filename}")
    else:
        print("No data fetched. DataFrame is empty.")


Fetching country data and creating DataFrame...

DataFrame Head:
  country_code country_name
0          ABW        Aruba
1          AFG  Afghanistan
2          AGO       Angola
3          AIA     Anguilla
4          ALB      Albania

Total countries fetched: 241

DataFrame saved to imf_countries.csv


In [25]:
import pandas as pd

# Load datasets
df_countries = pd.read_csv("imf_countries.csv")  # contains country_code and country_name
df_macro = pd.read_csv("imf_macro_indicators_long.csv")  # contains year, country, indicator, value

# Rename 'country' to 'country_code' to enable the join
df_macro = df_macro.rename(columns={"country": "country_code"})

# Right join on 'country_code'
df_combined = df_macro.merge(df_countries, on="country_code", how="left")

# Optional: reorder columns
df_combined = df_combined[["year", "country_code", "country_name", "indicator", "value"]]

# Save or display
df_combined.to_csv("imf_macro_with_names.csv", index=False)
print(df_combined.head())


   year country_code country_name                          indicator  \
0  1995          ABW        Aruba  Government expenditure (% of GDP)   
1  1996          ABW        Aruba  Government expenditure (% of GDP)   
2  1997          ABW        Aruba  Government expenditure (% of GDP)   
3  1998          ABW        Aruba  Government expenditure (% of GDP)   
4  1999          ABW        Aruba  Government expenditure (% of GDP)   

       value  
0  26.331776  
1  27.457660  
2  24.989031  
3  21.573601  
4  22.798358  


In [26]:
import pandas as pd

# Load data
df = pd.read_csv("imf_macro_with_names.csv")

# Filter years
df_filtered = df[(df["year"] >= 2005) & (df["year"] <= 2023)]

# Save filtered dataset
df_filtered.to_csv("imf_macro_2005_2023.csv", index=False)
print(df_filtered.head())


    year country_code country_name                          indicator  \
10  2005          ABW        Aruba  Government expenditure (% of GDP)   
11  2006          ABW        Aruba  Government expenditure (% of GDP)   
12  2007          ABW        Aruba  Government expenditure (% of GDP)   
13  2008          ABW        Aruba  Government expenditure (% of GDP)   
14  2009          ABW        Aruba  Government expenditure (% of GDP)   

        value  
10  24.476940  
11  24.826928  
12  22.619801  
13  25.359395  
14  26.887785  


In [32]:
import pandas as pd

# Load data
df = pd.read_csv("imf_macro_with_names.csv")

# Standardize peer country codes
peer_countries = {
    "IDN": "Indonesia",
    "PHL": "Philippines",
    "IND": "India",
    "VNM": "Vietnam",
    "COL": "Colombia",
    "MEX": "Mexico",
    "PER": "Peru",
    "ZAF": "South Africa",
    "ROU": "Romania",
    "HUN": "Hungary",
    "THA": "Thailand"
}

# Filter for years 2005–2023
df_filtered = df[df["year"].between(2005, 2023)]

# Filter only peer countries
df_peers = df_filtered[df_filtered["country_code"].isin(peer_countries.keys())].copy()
df_peers.loc[:, "country_name"] = df_peers["country_code"].map(peer_countries)

# Optional: Map standard names (in case they differ)
df_peers["country_name"] = df_peers["country_code"].map(peer_countries)

# Save to CSV
df_peers.to_csv("imf_macro_peers_filtered.csv", index=False)
print("Filtered peer data saved to imf_macro_peers_filtered.csv")


Filtered peer data saved to imf_macro_peers_filtered.csv


In [38]:
import requests
import pandas as pd
from tqdm import tqdm
from datetime import datetime
import time

# === SETTINGS ===
BASE_URL = "https://www.imf.org/external/datamapper/api/v1"
YEAR_RANGE = list(range(2005, 2024))  # 2005-2023 inclusive
INDICATORS = {
    "rev": "Government revenue (% of GDP)",
    "exp": "Government expenditure (% of GDP)",
    "prim_exp": "Primary expenditure (% of GDP)",
    "ie": "Interest on public debt (% of GDP)",
    "pb": "Primary balance (% of GDP)",
    "d": "Gross public debt (% of GDP)",
    "rgc": "Real GDP growth rate (%)",
    "rltir": "Real long-term bond yield (%)"
}
PEER_COUNTRIES = {
    "IDN": "Indonesia",
    "PHL": "Philippines",
    "IND": "India",
    "VNM": "Vietnam",
    "COL": "Colombia",
    "MEX": "Mexico",
    "PER": "Peru",
    "ZAF": "South Africa",
    "ROU": "Romania",
    "HUN": "Hungary",
    "THA": "Thailand"
}

# === MAIN FUNCTION ===
def fetch_and_process_data():
    """Fetch IMF data and process into long & wide formats"""
    print("Starting data processing...")
    
    # 1. Fetch data from IMF API
    all_data = []
    for code, label in tqdm(INDICATORS.items(), desc="Fetching indicators"):
        try:
            response = requests.get(f"{BASE_URL}/{code}", timeout=10)
            data = response.json().get("values", {}).get(code, {})
            
            for country, values in data.items():
                if country in PEER_COUNTRIES:
                    for year, value in values.items():
                        try:
                            year_int = int(year)
                            if year_int in YEAR_RANGE:
                                all_data.append({
                                    "year": year_int,
                                    "country_code": country,
                                    "country": PEER_COUNTRIES[country],
                                    "indicator": label,
                                    "value": float(value) if value else None
                                })
                        except (ValueError, TypeError):
                            continue
        except Exception as e:
            print(f"Skipping {code}: {str(e)}")
            continue
    
    if not all_data:
        raise Exception("No data fetched - check connection or API status")
    
    # 2. Create long format dataframe
    df_long = pd.DataFrame(all_data)
    df_long = df_long.sort_values(["country", "year", "indicator"])
    
    # 3. Create wide format dataframe
    df_wide = df_long.pivot_table(
        index=["country", "year"],
        columns="indicator",
        values="value"
    ).reset_index()
    
    # 4. Save both formats
    today = datetime.now().strftime("%Y%m%d")
    long_file = f"{today}_tfda_publicfinance_long.csv"
    wide_file = f"{today}_tfda_publicfinance_wide.csv"
    
    df_long.to_csv(long_file, index=False)
    df_wide.to_csv(wide_file, index=False)
    
    print(f"\n✅ Successfully saved:")
    print(f"- Long format: {long_file} ({len(df_long):,} rows)")
    print(f"- Wide format: {wide_file} ({len(df_wide):,} rows)")
    print("\nSample wide data:")
    print(df_wide.head())

# === EXECUTION ===
if __name__ == "__main__":
    try:
        fetch_and_process_data()
    except Exception as e:
        print(f"\n❌ Error: {e}")


Starting data processing...


Fetching indicators: 100%|██████████| 8/8 [00:26<00:00,  3.30s/it]


✅ Successfully saved:
- Long format: 20250705_tfda_publicfinance_long.csv (1,606 rows)
- Wide format: 20250705_tfda_publicfinance_wide.csv (209 rows)

Sample wide data:
indicator   country  year  Government expenditure (% of GDP)  \
0          Colombia  2005                          25.889331   
1          Colombia  2006                          28.441089   
2          Colombia  2007                          28.162299   
3          Colombia  2008                          28.428793   
4          Colombia  2009                          30.914217   

indicator  Government revenue (% of GDP)  Gross public debt (% of GDP)  \
0                              25.868560                     38.532495   
1                              27.448441                     35.969082   
2                              27.342520                     32.663327   
3                              28.470610                     32.382419   
4                              28.240170                     35.386764   



In [ ]:
!jupyter nbconvert --to html tfda_publicfinance_notebook.ipynb


[NbConvertApp] WARNING | pattern 'tfda_publicfinance_notebook.ipynb' matched no files
This application is used to convert notebook files (*.ipynb)
        to various other formats.


Options
The options below are convenience aliases to configurable class-options,
as listed in the "Equivalent to" description-line of the aliases.
To see all configurable class-options for some <cmd>, use:
    <cmd> --help-all

--debug
    set log level to logging.DEBUG (maximize logging output)
    Equivalent to: [--Application.log_level=10]
--show-config
    Show the application's configuration (human-readable format)
    Equivalent to: [--Application.show_config=True]
--show-config-json
    Show the application's configuration (json format)
    Equivalent to: [--Application.show_config_json=True]
--generate-config
    generate default config file
    Equivalent to: [--JupyterApp.generate_config=True]
-y
    Answer yes to any questions instead of prompting.
    Equivalent to: [--JupyterApp.answer_yes=Tru